# Module 04: Views & URLs

**Estimated Time:** 2 hours  
**Difficulty:** Beginner

---

## Learning Objectives

By the end of this module, you will:

- ✅ Understand Django's URL routing system
- ✅ Create function-based views (FBVs)
- ✅ Create class-based views (CBVs)
- ✅ Handle HTTP requests and responses
- ✅ Use URL parameters and path converters
- ✅ Work with generic views
- ✅ Implement proper URL organization

---

## Prerequisites

Before starting this module, ensure you've completed:
- ✅ Module 00: Setup & Introduction
- ✅ Module 01: Django Basics & First Project
- ✅ Module 02: Models & Databases
- ✅ Module 03: Django Admin

You should have:
- Django project with blog app
- Models created and migrated
- Sample data in admin

## 1. Understanding Views

In Django's MVT pattern, **Views** are the business logic layer that:
- Receive HTTP requests
- Process data (query database, etc.)
- Return HTTP responses

```
HTTP Request → URL Router → View → HTTP Response
                   ↓          ↓
              URL Pattern   Model/Template
```

### Two Types of Views

1. **Function-Based Views (FBVs)**: Simple Python functions
2. **Class-Based Views (CBVs)**: Python classes with methods

In [ ]:
import os
from pathlib import Path

# Setup paths
notebook_dir = Path.cwd()
project_path = notebook_dir.parent / "projects" / "myblog"
blog_app = project_path / "blog"
views_file = blog_app / "views.py"
urls_file = blog_app / "urls.py"
project_urls = project_path / "myblog" / "urls.py"

print(f"Project path: {project_path}")
print(f"Views file: {views_file}")
print(f"App URLs: {urls_file}")

## 2. Function-Based Views (FBVs)

FBVs are simple Python functions that take a request and return a response.

In [ ]:
# Create basic views
views_code = '''from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse, JsonResponse
from .models import Post, Category


def home(request):
    """Home page view - list recent published posts"""
    posts = Post.objects.filter(status='published').order_by('-publish_date')[:10]
    context = {
        'posts': posts,
        'title': 'Home',
    }
    return render(request, 'blog/home.html', context)


def post_detail(request, slug):
    """Post detail view - show single post"""
    post = get_object_or_404(Post, slug=slug, status='published')
    
    # Increment view count
    post.views += 1
    post.save()
    
    context = {
        'post': post,
        'title': post.title,
    }
    return render(request, 'blog/post_detail.html', context)


def category_posts(request, slug):
    """List posts in a category"""
    category = get_object_or_404(Category, slug=slug)
    posts = category.posts.filter(status='published').order_by('-publish_date')
    
    context = {
        'category': category,
        'posts': posts,
        'title': f'Category: {category.name}',
    }
    return render(request, 'blog/category_posts.html', context)


def about(request):
    """About page"""
    return render(request, 'blog/about.html', {'title': 'About'})
'''

with open(views_file, "w") as f:
    f.write(views_code)

print("✓ Function-based views created!")
print("\nViews created:")
print("  - home() - List recent posts")
print("  - post_detail(slug) - Show single post")
print("  - category_posts(slug) - Posts by category")
print("  - about() - About page")

### Understanding FBVs

#### Basic Structure
```python
def my_view(request):
    # 1. Process request
    # 2. Query data if needed
    # 3. Prepare context
    # 4. Return response
    return render(request, 'template.html', context)
```

#### Common Shortcuts
- `render()`: Render template with context
- `redirect()`: Redirect to another URL
- `get_object_or_404()`: Get object or return 404
- `get_list_or_404()`: Get queryset or return 404

#### Request Object
```python
request.method          # GET, POST, etc.
request.GET             # Query parameters
request.POST            # Form data
request.user            # Current user
request.path            # URL path
request.META            # HTTP headers
```

## 3. URL Configuration

URLs map web addresses to views. Let's create URL patterns for our views.

In [ ]:
# Create app URLs
app_urls_code = """from django.urls import path
from . import views

app_name = 'blog'

urlpatterns = [
    path('', views.home, name='home'),
    path('post/<slug:slug>/', views.post_detail, name='post_detail'),
    path('category/<slug:slug>/', views.category_posts, name='category_posts'),
    path('about/', views.about, name='about'),
]
"""

with open(urls_file, "w") as f:
    f.write(app_urls_code)

print("✓ App URLs created!")
print("\nURL patterns:")
print("  / → home")
print("  /post/<slug>/ → post_detail")
print("  /category/<slug>/ → category_posts")
print("  /about/ → about")

In [ ]:
# Update project URLs to include blog URLs
with open(project_urls, "r") as f:
    current_urls = f.read()

# Check if blog URLs already included
if "include('blog.urls')" not in current_urls:
    project_urls_code = """from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('', include('blog.urls')),
]
"""
    with open(project_urls, "w") as f:
        f.write(project_urls_code)
    print("✓ Project URLs updated to include blog URLs")
else:
    print("✓ Blog URLs already included in project")

### URL Patterns Explained

#### Basic Pattern
```python
path('blog/', views.blog_list, name='blog_list')
```

- **First arg**: URL pattern
- **Second arg**: View function
- **name**: URL name for reverse lookup

#### Path Converters
```python
<int:id>        # Integer (1, 42)
<str:name>      # String (default)
<slug:slug>     # Slug (my-post-title)
<uuid:id>       # UUID
<path:path>     # Path with slashes
```

#### App Namespacing
```python
# In blog/urls.py
app_name = 'blog'

# In templates
{% url 'blog:home' %}
{% url 'blog:post_detail' post.slug %}
```

## 4. Class-Based Views (CBVs)

CBVs provide reusable, object-oriented views. Let's add some class-based views.

In [ ]:
# Add class-based views
cbv_code = '''from django.shortcuts import render, get_object_or_404
from django.http import HttpResponse
from django.views.generic import ListView, DetailView, TemplateView
from .models import Post, Category


# Function-Based Views
def about(request):
    """About page"""
    return render(request, 'blog/about.html', {'title': 'About'})


# Class-Based Views
class HomeView(ListView):
    """Home page - list recent published posts"""
    model = Post
    template_name = 'blog/home.html'
    context_object_name = 'posts'
    paginate_by = 10
    
    def get_queryset(self):
        return Post.objects.filter(
            status='published'
        ).order_by('-publish_date')
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['title'] = 'Home'
        return context


class PostDetailView(DetailView):
    """Post detail view"""
    model = Post
    template_name = 'blog/post_detail.html'
    context_object_name = 'post'
    
    def get_queryset(self):
        return Post.objects.filter(status='published')
    
    def get_object(self):
        obj = super().get_object()
        # Increment view count
        obj.views += 1
        obj.save()
        return obj
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['title'] = self.object.title
        return context


class CategoryPostsView(ListView):
    """List posts in a category"""
    model = Post
    template_name = 'blog/category_posts.html'
    context_object_name = 'posts'
    paginate_by = 10
    
    def get_queryset(self):
        self.category = get_object_or_404(Category, slug=self.kwargs['slug'])
        return self.category.posts.filter(
            status='published'
        ).order_by('-publish_date')
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['category'] = self.category
        context['title'] = f'Category: {self.category.name}'
        return context


class AboutView(TemplateView):
    """About page"""
    template_name = 'blog/about.html'
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['title'] = 'About'
        return context
'''

with open(views_file, "w") as f:
    f.write(cbv_code)

print("✓ Class-based views created!")
print("\nCBVs created:")
print("  - HomeView (ListView)")
print("  - PostDetailView (DetailView)")
print("  - CategoryPostsView (ListView)")
print("  - AboutView (TemplateView)")

In [ ]:
# Update URLs to use CBVs
cbv_urls_code = """from django.urls import path
from . import views

app_name = 'blog'

urlpatterns = [
    path('', views.HomeView.as_view(), name='home'),
    path('post/<slug:slug>/', views.PostDetailView.as_view(), name='post_detail'),
    path('category/<slug:slug>/', views.CategoryPostsView.as_view(), name='category_posts'),
    path('about/', views.AboutView.as_view(), name='about'),
]
"""

with open(urls_file, "w") as f:
    f.write(cbv_urls_code)

print("✓ URLs updated to use class-based views")
print("\nNote: CBVs use .as_view() method in URLs")

### Generic Views Explained

#### ListView
Display list of objects
```python
class MyListView(ListView):
    model = MyModel
    template_name = 'my_list.html'
    context_object_name = 'items'
    paginate_by = 10
```

#### DetailView
Display single object
```python
class MyDetailView(DetailView):
    model = MyModel
    template_name = 'my_detail.html'
```

#### Other Generic Views
- **CreateView**: Create new object
- **UpdateView**: Update existing object
- **DeleteView**: Delete object
- **TemplateView**: Render template
- **RedirectView**: Redirect to URL

#### Key Methods
- `get_queryset()`: Customize query
- `get_context_data()`: Add context
- `get_object()`: Customize object retrieval

## 5. HTTP Responses

Django provides various response types.

In [ ]:
# Add more response types to views
response_views = """
# Additional response examples (add to views.py if needed)
from django.http import HttpResponse, JsonResponse, Http404
from django.shortcuts import redirect

# Simple text response
def simple_text(request):
    return HttpResponse("Hello, World!")

# JSON response
def api_posts(request):
    posts = Post.objects.filter(status='published').values(
        'title', 'slug', 'publish_date'
    )[:10]
    return JsonResponse(list(posts), safe=False)

# Redirect
def old_url(request):
    return redirect('blog:home')

# 404 Error
def custom_404(request):
    raise Http404("Page not found")

# Different status codes
def custom_response(request):
    return HttpResponse("Created", status=201)
"""

print("Response Types:")
print("\n1. HttpResponse - Basic text/HTML response")
print("2. JsonResponse - JSON data response")
print("3. redirect() - Redirect to another URL")
print("4. Http404 - Raise 404 error")
print("5. Custom status codes (201, 400, 500, etc.)")
print("\n" + response_views)

## 6. Advanced URL Patterns

Let's explore advanced URL features.

In [ ]:
# Advanced URL examples
advanced_urls = """# Advanced URL patterns examples

from django.urls import path, re_path, include
from . import views

app_name = 'blog'

urlpatterns = [
    # Basic patterns
    path('', views.HomeView.as_view(), name='home'),
    
    # URL with slug parameter
    path('post/<slug:slug>/', views.PostDetailView.as_view(), name='post_detail'),
    
    # URL with integer parameter
    path('post/id/<int:pk>/', views.PostDetailView.as_view(), name='post_by_id'),
    
    # Multiple parameters
    path('archive/<int:year>/<int:month>/', views.archive, name='archive'),
    
    # Optional trailing slash
    path('about/', views.AboutView.as_view(), name='about'),
    
    # Category posts
    path('category/<slug:slug>/', views.CategoryPostsView.as_view(), name='category_posts'),
]

# Regular expression patterns (for complex patterns)
# re_path(r'^post/(?P<year>[0-9]{4})/$', views.year_archive),
"""

print("Advanced URL Patterns:")
print(advanced_urls)

## 7. Testing Views

Let's verify our views and URLs are configured correctly.

In [ ]:
import subprocess

# Check for errors
result = subprocess.run(
    ["python", "manage.py", "check"], capture_output=True, text=True, cwd=project_path
)

print("System check:")
print(result.stdout)

if "no issues" in result.stdout.lower():
    print("\n✓ Views and URLs configured correctly!")
else:
    print("\n⚠ Issues found:", result.stderr)

## 8. FBV vs CBV Comparison

### When to Use FBVs
✅ Simple views with unique logic  
✅ Quick prototyping  
✅ Custom behavior not fitting generic patterns  
✅ When you're just learning Django  

### When to Use CBVs
✅ CRUD operations (Create, Read, Update, Delete)  
✅ Similar views with slight variations  
✅ When you need inheritance and mixins  
✅ Leveraging Django's generic views  

### Same View Both Ways

**FBV:**
```python
def post_list(request):
    posts = Post.objects.filter(status='published')
    return render(request, 'blog/posts.html', {'posts': posts})
```

**CBV:**
```python
class PostListView(ListView):
    model = Post
    template_name = 'blog/posts.html'
    
    def get_queryset(self):
        return Post.objects.filter(status='published')
```

## 9. Best Practices

### URL Design
1. **Use meaningful URLs**: `/blog/post/my-title/` not `/p/123/`
2. **Keep URLs lowercase**
3. **Use hyphens, not underscores**: `my-post` not `my_post`
4. **Be consistent** with trailing slashes
5. **Use namespaces** for apps

### View Design
1. **Keep views thin**: Move complex logic to models or services
2. **Use get_object_or_404**: Better than try/except
3. **Return early**: Handle errors first, main logic last
4. **Use decorators**: @login_required, @require_POST
5. **Follow DRY**: Don't repeat yourself

### Performance
1. **Optimize queries**: Use select_related, prefetch_related
2. **Cache expensive operations**
3. **Paginate large querysets**
4. **Avoid N+1 queries**

## 10. Common Patterns

### Search View
```python
def search(request):
    query = request.GET.get('q', '')
    posts = Post.objects.filter(
        Q(title__icontains=query) | Q(content__icontains=query),
        status='published'
    )
    return render(request, 'search.html', {'posts': posts, 'query': query})
```

### Pagination
```python
from django.core.paginator import Paginator

def post_list(request):
    posts = Post.objects.filter(status='published')
    paginator = Paginator(posts, 10)  # 10 per page
    page_number = request.GET.get('page')
    page_obj = paginator.get_page(page_number)
    return render(request, 'posts.html', {'page_obj': page_obj})
```

### Archive by Date
```python
def year_archive(request, year):
    posts = Post.objects.filter(
        publish_date__year=year,
        status='published'
    )
    return render(request, 'archive.html', {'posts': posts, 'year': year})
```

## 11. Troubleshooting

### Common Errors

**Error**: `Page not found (404)`  
**Solution**: Check URL pattern matches request path. Check app URLs are included in project URLs.

**Error**: `NoReverseMatch`  
**Solution**: Verify URL name is correct. Check you're using app namespace if defined.

**Error**: `TemplateDoesNotExist`  
**Solution**: Create the template file. Check TEMPLATES settings. Verify template path.

**Error**: `TypeError: as_view() takes 1 positional argument`  
**Solution**: CBVs need `.as_view()` in URLs: `MyView.as_view()`

**Error**: `AttributeError: 'function' object has no attribute 'as_view'`  
**Solution**: FBVs don't use `.as_view()`. Remove it from URL pattern.

## 12. Summary & Next Steps

### What We Accomplished

✅ Created function-based views (FBVs)  
✅ Created class-based views (CBVs)  
✅ Configured URL routing with path converters  
✅ Used generic views (ListView, DetailView, TemplateView)  
✅ Handled HTTP requests and responses  
✅ Implemented URL namespacing  
✅ Learned view best practices  

### Current Application Structure

```
URLs:                  Views:
/                  →   HomeView (list posts)
/post/<slug>/      →   PostDetailView (show post)
/category/<slug>/  →   CategoryPostsView (category posts)
/about/            →   AboutView (about page)
```

### What's Next

In **Module 05**, we'll:
- Create templates for our views
- Learn Django Template Language (DTL)
- Implement template inheritance
- Use template tags and filters
- Build our blog's HTML structure

### Quick Practice

Before moving on, try:
1. Add a new view for post archive by year
2. Create a search view
3. Add pagination to HomeView
4. Create a JSON API view

---

**Excellent work! You now understand Django's routing and view system. Continue to Module 05!** 🚀